In [2]:
from jupyter_dash import JupyterDash
import dash
import numpy as np
from dash import dcc, html, Input, Output, State, dash_table
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from PIL import Image
from sklearn import preprocessing

In [3]:
gps_merged = pd.read_csv('gps_merged.csv')
gps = pd.read_csv('gps_fixed.csv')
stop = pd.read_csv('stop_merged.csv')
stop_truck = pd.read_csv('stop_merged_truck.csv')
card = pd.read_csv('card_df_hour.csv')

In [4]:
gps_merged = gps_merged[gps_merged['id']!=28]
gps

,Unnamed: 0,Timestamp,id,lat,long,date,day_of_week,weekend,hour,time
0,0,2014-01-06 06:28:01,35,36.076225,24.874689,2014-01-06,0,False,6,Morning
1,1,2014-01-06 06:28:01,35,36.076220,24.874596,2014-01-06,0,False,6,Morning
2,2,2014-01-06 06:28:03,35,36.076211,24.874443,2014-01-06,0,False,6,Morning
3,3,2014-01-06 06:28:05,35,36.076217,24.874253,2014-01-06,0,False,6,Morning
4,4,2014-01-06 06:28:06,35,36.076214,24.874167,2014-01-06,0,False,6,Morning
...,...,...,...,...,...,...,...,...,...,...
685164,685164,2014-01-19 20:56:43,30,36.058110,24.902130,2014-01-19,6,True,20,Night
685165,685165,2014-01-19 20:56:47,30,36.058258,24.901774,2014-01-19,6,True,20,Night
685166,685166,2014-01-19 20:56:48,30,36.058296,24.901711,2014-01-19,6,True,20,Night
685167,685167,2014-01-19 20:56:49,30,36.058304,24.901620,2014-01-19,6,True,20,Night


In [5]:
types = dcc.Dropdown(
    id='types_dropdown',
    options=sorted(list(gps_merged['CurrentEmploymentType'].unique())),
    value=sorted(list(gps_merged['CurrentEmploymentType'].unique()))[0:1],
    multi=True)

time = dcc.Dropdown(
    id='time_dropdown',
    options=sorted(list(gps_merged['time'].unique())),
    value=sorted(list(gps_merged['time'].unique()))[0:1],
    multi=True)

weekend = dcc.RadioItems(
        id='weekend-radio',
        options=[
            {'label': 'True', 'value': True},
            {'label': 'False', 'value': False}],
        value=False)

name = dcc.Dropdown(
    id='name_dropdown',
    options = sorted(list(gps_merged['FullName'].unique())),
    value=sorted(list(gps_merged['FullName'].unique()))[0:1],
    multi=True)

date = dcc.Dropdown(
    id='date_dropdown',
    options = sorted(list(gps_merged['date'].unique())),
    value=sorted(list(gps_merged['date'].unique()))[0:1],
    multi=True)

truck_id = dcc.Dropdown(
    id='truck_id_dropdown',
    options = sorted(list(gps[gps['id']>=100].id.unique())),
    value=sorted(list(gps[gps['id']>=100].id.unique()))[0:1],
    multi=True)

select_all = dcc.Checklist(
        id='select-all',
        options=[{'label': 'Select All', 'value': 1}], 
        value=[])

In [6]:
img = Image.open("map-tourist.jpg")
img_width, img_height = img.size

lon_range = [24.825, 24.9095]
lat_range = [36.045, 36.0955]

graph = html.Div([
    dcc.Graph(
        id='map',
        figure={'layout': go.Layout(
                images=[{
                    'source': img,
                    'xref': "x",
                    'yref': "y",
                    'x': lon_range[0],
                    'y': lat_range[1],
                    'sizex': lon_range[1] - lon_range[0],
                    'sizey': lat_range[1] - lat_range[0],
                    'sizing': "stretch",
                    'opacity': 1,
                    'layer': "below",
                }],
                xaxis=dict(range=[lon_range[0], lon_range[1]]),
                yaxis=dict(range=[lat_range[0], lat_range[1]]),
                template="plotly_white",
                width=1800,
                height=1000,
            )
        }
    )
])

graph2 = html.Div([
    dcc.Graph(
        id='map2',
        figure={'layout': go.Layout(
                images=[{
                    'source': img,
                    'xref': "x",
                    'yref': "y",
                    'x': lon_range[0],
                    'y': lat_range[1],
                    'sizex': lon_range[1] - lon_range[0],
                    'sizey': lat_range[1] - lat_range[0],
                    'sizing': "stretch",
                    'opacity': 1,
                    'layer': "below",
                }],
                xaxis=dict(range=[lon_range[0], lon_range[1]]),
                yaxis=dict(range=[lat_range[0], lat_range[1]]),
                template="plotly_white",
                width=1800,
                height=1000,
            )
        }
    )
])

map_result=gps[gps['id']>=100]
le = preprocessing.LabelEncoder()

In [7]:
app = JupyterDash(__name__)
suppress_callback_exceptions=True

#################################################################################
map_result=gps[gps['id']>=100]
tab1_layout = html.Div([
    html.H1("Truck Driver Map"),
    truck_id,
    date,
    time,
    dcc.Graph(
            id='truck-plot',
            figure={
                'data': [
                    go.Scattergl(
                        x=map_result.long, y=map_result.lat, mode='markers', marker_color=map_result['id'],
                        hoverinfo='text', text=map_result['id'])],
                'layout': go.Layout(
                images=[{
                    'source': img,
                    'xref': "x",
                    'yref': "y",
                    'x': lon_range[0],
                    'y': lat_range[1],
                    'sizex': lon_range[1] - lon_range[0],
                    'sizey': lat_range[1] - lat_range[0],
                    'sizing': "stretch",
                    'opacity': 1,
                    'layer': "below",}],
                xaxis=dict(range=[lon_range[0], lon_range[1]]),
                yaxis=dict(range=[lat_range[0], lat_range[1]]),
                template="plotly_white",
                width=1800,
                height=1000)}),
    html.Div(dash_table.DataTable(id="truck_transaction"))])


# Callback
@app.callback(Output('truck-plot', 'figure'),
              [Input('date_dropdown', 'value'),
               Input('truck_id_dropdown', 'value'),
               Input('time_dropdown', 'value')
              ])



def update_figure(chosen_date,chosen_id,chosen_time):
    map_result=gps[(gps['id'].isin(chosen_id)) & (gps['date'].isin(chosen_date)) & (gps['time'].isin(chosen_time))]
    stop_result = stop_truck[(stop_truck['id'].isin(chosen_id)) & (stop_truck['date'].isin(chosen_date)) & (stop_truck['time'].isin(chosen_time))]
    locations=[go.Scattergl(
                    x=map_result['long'], 
                    y=map_result['lat'],
                    mode='markers',
                    marker_color=map_result['id'].astype("category"),
                    hoverinfo='text',
                    text=map_result['id'],
                    showlegend = False),
              go.Scatter(
                    x=stop_result['long'], 
                    y=stop_result['lat'],
                    mode='markers+text',
                    marker_color=stop_result['id'].astype("category"),
                    customdata=np.stack((stop_result['FullName'], stop_result['Timestamp']), axis=-1),
                    hovertemplate='<b>Stop : %{text}</b><br>Time:%{customdata[1]} <extra></extra>',
                    text=stop_result['stop'],
                    textposition='top center',
                    hoverinfo='text',
                    marker=dict(
                    size=25,
                    line=dict(
                    color='black',
                    width=3)),
                    textfont=dict(
                    family="Arial Black",
                    size=35,
                    color="black"),
                    showlegend = False)]
    return {
        'data': locations,
        'layout': go.Layout(
                images=[{
                    'source': img,
                    'xref': "x",
                    'yref': "y",
                    'x': lon_range[0],
                    'y': lat_range[1],
                    'sizex': lon_range[1] - lon_range[0],
                    'sizey': lat_range[1] - lat_range[0],
                    'sizing': "stretch",
                    'opacity': 1,
                    'layer': "below",
                }],
                xaxis=dict(range=[lon_range[0], lon_range[1]]),
                yaxis=dict(range=[lat_range[0], lat_range[1]]),
                template="plotly_white",
                width=1800,
                height=1000,
            )
        
    }

@app.callback(Output('truck_transaction', 'data'),
              [Input('date_dropdown', 'value'),
              Input('time_dropdown', 'value')])


def update_truck_table(chosen_date,chosen_time):
    card_result=card[(card['timestamp'].isin(chosen_date)) & (card['time'].isin(chosen_time) | card['time'].isnull()) & (card['CurrentEmploymentTitle']=='Truck Driver')]
    show = card_result.filter(items=['timestamp_full', 'CarId', 'FullName','location','price','cc_flag','loyal_flag'])
    return show.to_dict('records')


##############################################################################
tab2_layout = html.Div([
    html.H1("General Employee Movement"),
    html.P("Employment Type:"),
    types,
    html.P("Time of the Day"),
    time,
    html.P("Weekend"),
    weekend,
    graph,
     html.Div(dash_table.DataTable(id="table_data"))
])

# Callback

@app.callback(Output('map', 'figure'),
              [Input('types_dropdown', 'value'),
               Input('time_dropdown', 'value'),
              Input('weekend-radio', 'value')])


def update_figure(chosen_types,chosen_time,weekend):
    map_result=gps_merged[(gps_merged['time'].isin(chosen_time)) & (gps_merged['weekend']==weekend) & 
                          (gps_merged['CurrentEmploymentType'].isin(chosen_types))]
    map_result['label'] = le.fit_transform(map_result['CurrentEmploymentType'])
    
    locations=[go.Scattergl(
                    x=map_result['long'], 
                    y=map_result['lat'],
                    mode='markers',
                    marker_color=map_result['label'].astype("category"),
                    customdata=np.stack((map_result['CurrentEmploymentType'], map_result['time']), axis=-1),
                    hovertemplate='<b>Full Name : %{text}</b><br>Type:%{customdata[0]} <extra></extra>',
                    text= map_result['FullName'],
                    showlegend = False)]
    return {
        'data': locations,
        'layout': go.Layout(
                images=[{
                    'source': img,
                    'xref': "x",
                    'yref': "y",
                    'x': lon_range[0],
                    'y': lat_range[1],
                    'sizex': lon_range[1] - lon_range[0],
                    'sizey': lat_range[1] - lat_range[0],
                    'sizing': "stretch",
                    'opacity': 1,
                    'layer': "below",
                }],
                xaxis=dict(range=[lon_range[0], lon_range[1]]),
                yaxis=dict(range=[lat_range[0], lat_range[1]]),
                template="plotly_white",
                width=1800,
                height=1000,
            )
        
    }

@app.callback(Output('table_data', 'data'),
              [Input('types_dropdown', 'value'),
               Input('time_dropdown', 'value'),
              Input('weekend-radio', 'value')])

def update_table(chosen_types,chosen_time,weekend):
    map_result=gps_merged[(gps_merged['time'].isin(chosen_time)) & (gps_merged['weekend']==weekend) & 
                          (gps_merged['CurrentEmploymentType'].isin(chosen_types))]
    show = map_result.filter(items=['Timestamp', 'id', 'FullName', 'CurrentEmploymentType', 'CurrentEmploymentTitle'])
    return show.to_dict('records')
#####################################################################################

tab3_layout = html.Div([
    html.H1("Individual Employee Movement"),
    html.P("Employee Name:"),
    name,
    html.P("Date"),
    date,
    select_all,
    html.P("Time of the Day"),
    time,
    graph2,
    html.H1("Card Transaction Table"),
    html.Div(dash_table.DataTable(id="table_data2", style_cell = {
                'font_family': 'Times New Roman',
                'font_size': '20px',
                'text_align': 'center'
            },))
    ])

# Callback

    
@app.callback(
    Output('date_dropdown', 'value'),
    [Input('select-all', 'value')],
    [State('date_dropdown', 'options'),
     State('select-all', 'options')])
def test(selected, options_1, options_2):
    if len(selected) > 0:
        return options_1
    else:
        return []

    
@app.callback(Output('map2', 'figure'),
              [Input('name_dropdown', 'value'),
               Input('date_dropdown', 'value'),
              Input('time_dropdown', 'value')])


def update_figure(chosen_name,chosen_date,chosen_time):
    map_result=gps_merged[(gps_merged['FullName'].isin(chosen_name)) & (gps_merged['date'].isin(chosen_date)) & 
                          (gps_merged['time'].isin(chosen_time))]
    chosen_id = map_result.id.unique()
    stop_result = stop[(stop['id'].isin(chosen_id)) & (stop['date'].isin(chosen_date)) & (stop['time'].isin(chosen_time))]
    
    
    locations=[go.Scattergl(
                    x=map_result['long'], 
                    y=map_result['lat'],
                    mode='markers',
                    marker_color=map_result['id'].astype("category"),
                    hoverinfo='text',
                    text=map_result['FullName'],
                    showlegend = False),
              go.Scatter(
                    x=stop_result['long'], 
                    y=stop_result['lat'],
                    mode='markers+text',
                    marker_color=stop_result['id'].astype("category"),
                    customdata=np.stack((stop_result['FullName'], stop_result['Timestamp']), axis=-1),
                    hovertemplate='<b>Stop : %{text}</b><br>Name:%{customdata[0]}<br>Time:%{customdata[1]} <extra></extra>',
                    text=stop_result['stop'],
                    textposition='top center',
                    hoverinfo='text',
                    marker=dict(
                    size=25,
                    line=dict(
                    color='black',
                    width=3)),
                    textfont=dict(
                    family="Arial Black",
                    size=35,
                    color="black"),
                    showlegend = False)]
    
    
    return {
        'data': locations,
        'layout': go.Layout(
                images=[{
                    'source': img,
                    'xref': "x",
                    'yref': "y",
                    'x': lon_range[0],
                    'y': lat_range[1],
                    'sizex': lon_range[1] - lon_range[0],
                    'sizey': lat_range[1] - lat_range[0],
                    'sizing': "stretch",
                    'opacity': 1,
                    'layer': "below",
                }],
                xaxis=dict(range=[lon_range[0], lon_range[1]]),
                yaxis=dict(range=[lat_range[0], lat_range[1]]),
                template="plotly_white",
                width=1800,
                height=1000,
            )
        
    }

@app.callback(Output('table_data2', 'data'),
              [Input('name_dropdown', 'value'),
               Input('date_dropdown', 'value'),
              Input('time_dropdown', 'value')])


def update_table(chosen_name,chosen_date,chosen_time):
    card_result=card[(card['FullName'].isin(chosen_name)) & (card['timestamp'].isin(chosen_date)) & 
                          (card['time'].isin(chosen_time) | card['time'].isnull())]
    show = card_result.filter(items=['timestamp_full', 'CarId', 'FullName','location','price','cc_flag','loyal_flag'])
    return show.to_dict('records')

#####################################################################################

# Create the app layout with the tabs
app.layout = html.Div([
    dcc.Tabs(id="tabs", value='tab-1', children=[
        dcc.Tab(label='Truck Route', value='tab-1'),
        dcc.Tab(label='General Movement', value='tab-2'),
        dcc.Tab(label='Individual Movement', value='tab-3'),
    ]),
    html.Div(id='tabs-content')
])

# Define callback to switch the content based on the selected tab
@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return tab1_layout
    elif tab == 'tab-2':
        return tab2_layout
    elif tab == 'tab-3':
        return tab3_layout

/Users/williamtandio/opt/anaconda3/envs/dst1/lib/python3.8/site-packages/dash/dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


In [8]:
app.run_server(dev_tools_ui=False,port=8051)

Dash app running on http://127.0.0.1:8051/


/var/folders/zj/rsdj4tn13qncnnvct08g82nm0000gn/T/ipykernel_21251/2954570511.py:137: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

